In [1]:
%matplotlib widget

In [2]:
import numpy as np
import scipy.optimize as sopt
import matplotlib.pyplot as plt
import pandas as pd
from pydae.tools import get_v,get_i,get_s
import pydae.ssa as ssa

In [3]:
from cigre_lv_res_vsg import cigre_lv_res_vsg_class

In [4]:
syst =cigre_lv_res_vsg_class()
syst.Dt = 0.01
syst.decimation = 1
syst.update()

events = [{ # CTRL4-3-0
           't_end':0.0, 
           'K_f_sec':0.001,
           'K_q_G10':0.5, 'K_q_G14':0.5,'D_G10':1,'D_G14':1,
           'R_v_G10':0.01,'X_v_G10':0.1,'R_v_G14':0.01,'X_v_G14':0.1,
           'K_f_G10':5,'K_f_G14':5,
           'K_vpoi_G10':50,'K_vpoi_G14':50,
           'K_phi_G10':1e-3,'K_phi_G14':1e-3,
           'p_r_G10':0.0,'q_r_G10':0.0, 
           'p_r_G14':0.0,'q_r_G14':0.0, #'R_12':0.01, 'R_23':0.01, 'L_12':1e-3, 'L_23':1e-3, 'C_12':1e-6, 'C_23':1e-6,'R_t_1':0.01,'R_t_2':0.01,'L_t_1':1e-3,'L_t_2':1e-3
           'v_s_ref_G10':1.0,'v_s_ref_G14':1.0,'omega_ref_G10':1.0,'omega_ref_G14':1.0,
          },
          {'t_end':1.0}, 
          {'t_end':6.0},
          {'t_end':15.0,'p_r_G10':0.05,'p_r_G14':0.05}
          ]

loads_0 = [
        {"bus": "R01", "kVA":  1.0, "pf": 0.95, "T_i":0.01,"I_max":434.78},
        {"bus": "R11", "kVA": 15.0, "pf": 0.95, "T_i":0.01,"I_max":32.6},
        {"bus": "R15", "kVA": 52.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R16", "kVA": 55.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R17", "kVA": 35.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R18", "kVA": 47.0, "pf": 0.95, "T_i":0.01,"I_max":120}
        ]

loads_1 = [
        {"bus": "R01", "kVA":  1.0, "pf": 0.95, "T_i":0.01,"I_max":434.78},
        {"bus": "R11", "kVA": 15.0, "pf": 0.95, "T_i":0.01,"I_max":32.6},
        {"bus": "R15", "kVA": 52.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R16", "kVA": 55.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R17", "kVA": 35.0, "pf": 0.95, "T_i":0.01,"I_max":120},
        {"bus": "R18", "kVA": 70.0, "pf": 0.95, "T_i":0.01,"I_max":120}
        ]

# makes all loads zero for initilization
loads_dict = {}
for load in loads_0:
    p = f"p_{load['bus']}_ref"
    q = f"q_{load['bus']}_ref"
    events[0][p] = 0.0
    events[0][q] = 0.0    

syst.sopt_root_jac = True
syst.initialization_tol = 1e-2
syst.initialize(events,xy0=1)
syst.sopt_root_jac = True

syst.initialization_tol = 1e-12
syst.xy_prev[syst.x_list.index('phi_G10')]  = 0.0
syst.xy_prev[syst.x_list.index('phi_G14')]  = 0.0

# assign initial loads
for load in loads_0:
    p_ref_name = f"p_{load['bus']}_ref"
    q_ref_name = f"q_{load['bus']}_ref"
    s = load['kVA']*1000
    p = s*load['pf']
    q = np.sign(load['pf'])*(s**2 - p**2)**0.5
    events[0][p_ref_name] = p
    events[0][q_ref_name] = q
    events[1][p_ref_name] = p
    events[1][q_ref_name] = q
    
# assign step 1 loads
for load in loads_1:
    p_ref_name = f"p_{load['bus']}_ref"
    q_ref_name = f"q_{load['bus']}_ref"
    s = load['kVA']*1000
    p = s*load['pf']
    q = np.sign(load['pf'])*(s**2 - p**2)**0.5
    events[2][p_ref_name] = p
    events[2][q_ref_name] = q
    
    
events[0].update(loads_dict)    
syst.initialize(events,xy0='prev')
ssa.eval_A(syst);

In [5]:
syst.simulate(events,xy0='prev');

In [6]:
fig, axes = plt.subplots(nrows=4, figsize=(7, 10), dpi=80)


#axes[0].plot(syst.T, syst.get_values('i_sD_G10'), label=f'i_sD_G10')
#axes[0].plot(syst.T, syst.get_values('i_sQ_G10'), label=f'i_sQ_G10')

#axes[0].plot(syst.T, syst.get_values('i_sD_G14'), label=f'i_sD_G14')
#axes[0].plot(syst.T, syst.get_values('i_sQ_G14'), label=f'i_sQ_G14')


axes[0].plot(syst.T, syst.get_values('p_s_pu_G10')*200, label=f'p_s_pu_G10')
axes[0].plot(syst.T, syst.get_values('p_s_pu_G14')*200, label=f'p_s_pu_G14')

axes[1].plot(syst.T, syst.get_values('q_s_pu_G10')*200, label=f'q_s_pu_G10')
axes[1].plot(syst.T, syst.get_values('q_s_pu_G14')*200, label=f'q_s_pu_G14')

axes[2].plot(syst.T, syst.get_values('omega_v_G10'), label=f'omega_v_G10')
axes[2].plot(syst.T, syst.get_values('omega_v_G14'), label=f'omega_v_G14')

for ax in axes:
    ax.legend(loc='upper right')
    ax.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [63]:
loads_kva = [item['kVA'] for item in loads] 

NameError: name 'loads' is not defined

In [64]:
ssa.damp_report(syst).round(3)

,Real,Imag,Freq.,Damp
Mode 1,-1.000,0.000,0.000,1.000
Mode 2,-1.000,0.000,0.000,1.000
Mode 3,-198.027,0.000,0.000,1.000
Mode 4,-89.947,59.736,9.507,0.833
Mode 5,-89.947,-59.736,9.507,0.833
Mode 6,-103.505,0.000,0.000,1.000
Mode 7,-96.254,0.000,0.000,1.000
Mode 8,-101.250,0.000,0.000,1.000
Mode 9,-98.749,0.000,0.000,1.000
Mode 10,-100.687,0.000,0.000,1.000


In [78]:
np.sum(np.array(loads_kva))

205.0

In [9]:
get_s(syst,'R01')

(-190000.00000000006-62449.97998398403j)

In [11]:
fig, axes = plt.subplots(nrows=4, figsize=(7, 10), dpi=50)


#for ig in range(1,6):
#axes[0].plot(T, (Z[:,0]-, label=f'$\Delta f_{{coi}}$')

bus = 'R18'
v_d = Y[:,syst.y_list.index(f'v_{bus}_d')]
v_q = Y[:,syst.y_list.index(f'v_{bus}_q')]
axes[0].plot(T, np.abs(v_d+1j*v_q)*np.sqrt(3/2), label=f'{bus}: $V$')

bus = 'R00'
v_d = Z[:,0]
v_q = Z[:,1]

i_d = Y[:,syst.y_list.index(f'i_{bus}_d')]
i_q = Y[:,syst.y_list.index(f'i_{bus}_q')]

p = 3/2*(i_d*v_d + i_q*v_q)
q = 3/2*(i_q*v_d - i_d*v_q)

axes[1].plot(T, p/1000, label=f'{bus}: $p$')
axes[1].plot(T, q/1000, label=f'{bus}: $p$')

v_r18_d = Y[-1,syst.y_list.index('v_R18_d')]
v_r18_q = Y[-1,syst.y_list.index('v_R18_q')]
v_r18_m = np.abs(v_r18_d+1j*v_r18_q)*np.sqrt(3/2)
print(f'V_R18 = [{v_r18_d:0.2f},{v_r18_q:0.2f}, |V_R18| = {v_r18_m:0.3f}]' )
print(f'V_R18 = [{p[-1]:0.2f},{q[-1]:0.2f}, |V_R18| = {v_r18_m:0.3f}]' )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

V_R18 = [-10.66,301.58, |V_R18| = 369.586]
V_R18 = [398270.37,144564.01, |V_R18| = 369.586]


In [29]:
buses = [{"bus":"R00"},{"bus":"R01"},{"bus":"R02"},{"bus":"R03"},{"bus":"R04"},{"bus":"R05"},{"bus":"R06"},
        {"bus":"R07"},{"bus":"R08"},{"bus":"R09"},{"bus":"R10"},{"bus":"R11"},{"bus":"R12"},
        {"bus":"R13"},{"bus":"R14"},{"bus":"R15"},{"bus":"R16"},{"bus":"R17"},{"bus":"R18"}]

In [30]:
buses_list = [bus['bus'] for bus in buses]

In [33]:
ds_v = [1.0,0.9808973,0.9722749,0.9636541,0.9556469,0.9498654,0.9440847,0.9406105,0.9371365,
0.9336628,0.9316682,0.9612911,0.9455948,0.9355475,0.925505,0.9154676,0.935177,
0.9279505,0.923964]

In [129]:
it = 0
p_total = 0.0
header = f"{'Bus':6s}|  {'v_m':5s}|  {'error':7s}|{'p':7s}| {'q':7s}"
print(header)
print(':-----|:------|--------:|------:|---------:')
for bus in buses:
    #if bus['bus']=='R00': continue

    v_m = get_v(syst,bus['bus'])
    s = get_s(syst,bus['bus'])
    
    error = v_m/400 -ds_v[it] 
    
    print(f"{bus['bus']:6s}| {v_m:5.1f} | {100*np.abs(error):6.4f}% |{s.real/1000:7.2f}| {s.imag/1000:7.2f}")
    p_total += s.real
    it+=1

Bus   |  v_m  |  error  |p      | q      
:-----|:------|--------:|------:|---------:
R00   | 400.0 | 0.0000% | 398.27|  144.56
R01   | 392.4 | 0.0000% |-190.00|  -62.45
R02   | 388.9 | 0.0000% |   0.00|   -0.00
R03   | 385.5 | 0.0000% |   0.00|   -0.00
R04   | 382.3 | 0.0000% |   0.00|   -0.00
R05   | 379.9 | 0.0000% |   0.00|   -0.00
R06   | 377.6 | 0.0000% |   0.00|   -0.00
R07   | 376.2 | 0.0000% |   0.00|   -0.00
R08   | 374.9 | 0.0000% |   0.00|   -0.00
R09   | 373.5 | 0.0000% |   0.00|   -0.00
R10   | 372.7 | 0.0000% |   0.00|   -0.00
R11   | 384.5 | 0.0000% | -14.25|   -4.68
R12   | 378.2 | 0.0000% |   0.00|   -0.00
R13   | 374.2 | 0.0000% |   0.00|   -0.00
R14   | 370.2 | 0.0000% |   0.00|   -0.00
R15   | 366.2 | 0.0000% | -49.40|  -16.24
R16   | 374.1 | 0.0000% | -52.25|  -17.17
R17   | 371.2 | 0.0000% | -33.25|  -10.93
R18   | 369.6 | 0.0000% | -44.65|  -14.68


In [71]:
data = {'bus':[bus['bus'] for bus in buses],
        'v_m':[get_v(syst,bus['bus']) for bus in buses],
        'p':[get_s(syst,bus['bus']).real/1000 for bus in buses],
        'q':[get_s(syst,bus['bus']).imag/1000 for bus in buses]}
df = pd.DataFrame(data=data)
df.set_index('bus')

df['v_m'] = df['v_m'].apply(lambda x: "{:,.1f}".format(x))
df['p'] = df['p'].apply(lambda x: "{:,.1f}".format(x))
df['q'] = df['q'].apply(lambda x: "{:,.1f}".format(x))
df = df.set_index('bus')
df

NameError: name 'buses' is not defined

In [89]:
import sympy as sym
v_d,v_q,i_d,i_q = sym.symbols('v_d,v_q,i_d,i_q', real=True)
i_p_ref,i_q_ref = sym.symbols('i_p_ref,i_q_ref ', real=True)
v_dq = v_q +1j*v_d
i_dq = i_q +1j*i_d
s = 3/2*v_dq*np.conj(i_dq)

v_m = sym.sqrt(v_d**2+v_q**2)

g_p = -i_p_ref + 3/2*(i_d*v_d + i_q*v_q)/v_m
g_q = -i_q_ref + 3/2*(i_q*v_d - i_d*v_q)/v_m

In [90]:
sol = sym.solve([g_p,g_q],[i_d,i_q])

In [91]:
print(sol[i_q])

0.666666666666667*(i_p_ref*v_q + i_q_ref*v_d)/sqrt(v_d**2 + v_q**2)


In [92]:
i_d_ref = 2/3*(i_p_ref*v_d - i_q_ref*v_q)/sqrt(v_d^2 + v_q^2)
i_q_ref = 2/3*(i_p_ref*v_q + i_q_ref*v_d)/sqrt(v_d^2 + v_q^2)


NameError: name 'sqrt' is not defined

In [11]:
syst.u_run

AttributeError: 'cigre_lv_res_vsg_class' object has no attribute 'u_run'